Consider the following minimization problem ([reference](https://alphaville.github.io/optimization-engine/docs/example_rosenbrock_py))
$$
\begin{align}
    \operatorname*{Minimize}_{\|u\|\leq r}& \sum_{i=1}^{n_u - 1} b (u_{i+1} - u_{i}^2)^2 + (a-u_i)^2
    \\
    \text{subject to: }& 1.5 u_1 - u_2 = 0
    \\
    &u_3 - u_4 + 0.1 \leq 0
    \end{align}
$$
The parameter vector is $p=(a,b)$. Let us generate a parametric optimiser with $n_u=5$.

In [1]:
import opengen as og
import casadi.casadi as cs

# Build parametric optimizer
# ------------------------------------
u = cs.SX.sym("u", 5)  # decision variables
p = cs.SX.sym("p", 2)  # parameters, p = (a, b)

# cost function:
phi = og.functions.rosenbrock(u, p)

# constraints:
c = cs.vertcat(1.5 * u[0] - u[1],
               cs.fmax(0.0, u[2] - u[3] + 0.1))

# simple bounds on decision variables:
bounds = og.constraints.Ball2(None, 1.5)

# problem formulation
problem = og.builder.Problem(u, p, phi) \
    .with_penalty_constraints(c)        \
    .with_constraints(bounds)

In [ ]:
# Configure and build
# (This might take a while)
# ---------------------------------
build_config = og.config.BuildConfiguration()      \
    .with_build_directory("optimizers")            \
    .with_build_mode(og.config.BuildConfiguration.DEBUG_MODE)  \
    .with_tcp_interface_config()
meta = og.config.OptimizerMeta()                   \
    .with_optimizer_name("my_optimizer")
solver_config = og.config.SolverConfiguration()    \
    .with_tolerance(1e-5)                          \
    .with_delta_tolerance(1e-4)                    \
    .with_initial_penalty(1e3)                     \
    .with_penalty_weight_update_factor(5)
builder = og.builder.OpEnOptimizerBuilder(problem, meta,
                                          build_config, solver_config)
builder.build()

In [ ]:
# Start the optimisation server
# ----------------------------------
mng = og.tcp.OptimizerTcpManager('optimizers/my_optimizer')
mng.start()

In [ ]:
# Call the server
# ----------------------------------
response = mng.call([1.0, 50.0])
if response.is_ok():
    # Solver returned a solution
    solution_data = response.get()
    u_star = solution_data.solution
    exit_status = solution_data.exit_status
    solver_time = solution_data.solve_time_ms
    print("u_star = ", u_star)
    print("solved in = ", solver_time, "ms")
    

In [8]:
# Kill the server
# ----------------------------------
mng.kill()

u_star =  [0.0215392358885952, 0.032299327741906976, -0.031339427619074, 0.06862231901442284, 0.004430167436941514]
solved in =  1.6935 ms
